In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split
import os

In [17]:
def Image_Forgery_Predictor_Model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        Conv2D(32, (3, 3), activation='relu'),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dense(2, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [18]:
def JPEGCompression(image, quality):
    # Simulación de compresión JPEG, puedes utilizar librerías como PIL para una compresión real
    return tf.image.adjust_jpeg_quality(image, jpeg_quality=quality)

def preprocess_image(image):
    image_resized = tf.image.resize(image, (128, 128))
    image_compressed = JPEGCompression(image_resized, quality=75)
    image_diff = tf.subtract(image_resized, image_compressed)
    return image_diff.numpy()


In [19]:
def train_model(images, labels, epochs=10):
    model = Image_Forgery_Predictor_Model()
    training_data = []
    for image in images:
        diff_image = preprocess_image(image)
        training_data.append(diff_image)
    training_data = np.array(training_data)
    labels = np.array(labels)
    
    # Dividir los datos en conjuntos de entrenamiento y validación
    X_train, X_val, y_train, y_val = train_test_split(training_data, labels, test_size=0.2, random_state=42)
    
    # Convertir las etiquetas a una representación binaria
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=2)
    
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs)
    return model


In [20]:
def predict_image(model, input_image):
    input_diff_image = preprocess_image(input_image)
    input_diff_image = np.expand_dims(input_diff_image, axis=0)  # Añadir dimensión de lote
    predicted_label = model.predict(input_diff_image)
    if predicted_label[0][0] > predicted_label[0][1]:
        return "Tampered"
    else:
        return "Untampered"


In [21]:
# Path to your CASIA2 dataset
PATH = '../data/CASIA2'

# Directories for authentic and tampered images
authentic_dir = os.path.join(PATH, 'Au')
tampered_dir = os.path.join(PATH, 'Tp')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

def get_file_list_and_labels(directory, label):
    file_list = []
    labels = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                file_list.append(os.path.join(root, file))
                labels.append(label)
    return file_list, labels

# Get file lists and labels
authentic_files, authentic_labels = get_file_list_and_labels(authentic_dir, 0)
tampered_files, tampered_labels = get_file_list_and_labels(tampered_dir, 1)

# Combine authentic and tampered data
all_files = authentic_files + tampered_files
all_labels = authentic_labels + tampered_labels

# Split the data into training and validation sets
train_files, val_files, train_labels, val_labels = train_test_split(
    all_files, all_labels, test_size=0.2, random_state=42, stratify=all_labels
)

def create_dataset(file_list, labels):
    labels = np.array(labels)
    ds = tf.data.Dataset.from_tensor_slices((file_list, labels))
    ds = ds.map(lambda x, y: (tf.io.read_file(x), y))
    ds = ds.map(lambda x, y: (tf.image.decode_jpeg(x, channels=3), y))
    ds = ds.map(lambda x, y: (tf.image.resize(x, IMG_SIZE), y))
    return ds.shuffle(1000).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

# Create training and validation datasets
train_dataset = create_dataset(train_files, train_labels)
validation_dataset = create_dataset(val_files, val_labels)

In [22]:
def load_tif_image(file_path):
    # Cargar la imagen usando Pillow
    image = Image.open(file_path)
    # Convertir la imagen a un arreglo numpy
    image_array = np.array(image)
    # Convertir el arreglo numpy a un tensor de TensorFlow
    return tf.convert_to_tensor(image_array)

def load_images_from_directory(directory_path):
    images = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".tif") or filename.endswith(".tiff"):
            image_path = os.path.join(directory_path, filename)
            image_tensor = load_tif_image(image_path)
            resized = tf.image.resize(image_tensor, IMG_SIZE)
            images.append(image_tensor)
        elif filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            image_path = os.path.join(directory_path, filename)
            image_tensor = tf.io.read_file(image_path)
            image_tensor = tf.image.decode_image(image_tensor)
            images.append(image_tensor)
    return images

# Directorio donde se encuentran las imágenes
directory_path = '../data/CASIA2/Tp'

# Cargar las imágenes
images = load_images_from_directory(directory_path)

# Imprimir la cantidad de imágenes cargadas
print(f'Se han cargado {len(images)} imágenes.')

NameError: name 'Image' is not defined

In [ ]:
# Supongamos que tenemos un conjunto de imágenes `images` y sus etiquetas `labels`
# images = [...]  # Lista de imágenes (numpy arrays)
# labels = [...]  # Lista de etiquetas correspondientes (0 para no manipuladas, 1 para manipuladas)

# Entrenar el modelo
model = train_model(images, labels, epochs=10)

# Predicción de una imagen de entrada
# input_image = [...]  # Una imagen de entrada (numpy array)
result = predict_image(model, input_image)
print("Resultado de la predicción:", result)


NameError: name 'images' is not defined